# Sandbox - Exploring IfcWindows Structure

In [12]:
import ifcopenshell
import ifcopenshell.util.element as Element

In [13]:
# Load the IFC file
ifc_file = ifcopenshell.open('../OpeningElements_2x3.ifc')

In [14]:
# Get all IfcWindow elements
windows = ifc_file.by_type('IfcWindow')
print(f"Total number of windows: {len(windows)}")

Total number of windows: 13


In [15]:
# Examine the first window in detail
if windows:
    window = windows[0]
    print(f"Window GlobalId: {window.GlobalId}")
    print(f"Window Name: {window.Name}")
    print(f"Window Type: {window.is_a()}")
    print(f"\nAll attributes:")
    print("-" * 50)
    for i, attr in enumerate(window):
        attr_name = window.attribute_name(i)
        print(f"{attr_name}: {attr}")

Window GlobalId: 3V_$GkZXXCGeUhpQedgVTe
Window Name: Window_Wood-AL_Openable_Double:BxH 20x14 utan bleck:107943
Window Type: IfcWindow

All attributes:
--------------------------------------------------
GlobalId: 3V_$GkZXXCGeUhpQedgVTe
OwnerHistory: #18=IfcOwnerHistory(#17,#2,$,.NOCHANGE.,$,$,$,1771156756)
Name: Window_Wood-AL_Openable_Double:BxH 20x14 utan bleck:107943
Description: None
ObjectType: Window_Wood-AL_Openable_Double:BxH 20x14 utan bleck
ObjectPlacement: #17405=IfcLocalPlacement(#17399,#17404)
Representation: #1930=IfcProductDefinitionShape($,$,(#1929))
Tag: 107943
OverallHeight: 1494.0000000001185
OverallWidth: 2080.9999999999977


In [16]:
# Get property sets for the first window
if windows:
    window = windows[0]
    psets = Element.get_psets(window)
    print("Property Sets:")
    print("=" * 50)
    for pset_name, properties in psets.items():
        print(f"\n{pset_name}:")
        for prop_name, prop_value in properties.items():
            if prop_name != 'id':
                print(f"  {prop_name}: {prop_value}")

Property Sets:

Window_Wood-AL_Openable_Double:BxH 20x14 utan bleck:107943:


In [17]:
# Examine all windows summary
print("Summary of all windows:")
print("=" * 80)
for i, window in enumerate(windows, 1):
    print(f"\nWindow {i}:")
    print(f"  GlobalId: {window.GlobalId}")
    print(f"  Name: {window.Name}")
    print(f"  Description: {window.Description}")
    print(f"  ObjectType: {window.ObjectType}")
    print(f"  Tag: {window.Tag}")
    
    # Get window type if available
    if hasattr(window, 'IsDefinedBy'):
        for definition in window.IsDefinedBy:
            if definition.is_a('IfcRelDefinesByType'):
                window_type = definition.RelatingType
                print(f"  Window Type Name: {window_type.Name}")

Summary of all windows:

Window 1:
  GlobalId: 3V_$GkZXXCGeUhpQedgVTe
  Name: Window_Wood-AL_Openable_Double:BxH 20x14 utan bleck:107943
  Description: None
  ObjectType: Window_Wood-AL_Openable_Double:BxH 20x14 utan bleck
  Tag: 107943
  Window Type Name: Window_Wood-AL_Openable_Double:BxH 20x14 utan bleck

Window 2:
  GlobalId: 3V_$GkZXXCGeUhpQedgVIN
  Name: Window_Wood-AL_Fixed_Double:wxh 20x05:108120
  Description: None
  ObjectType: Window_Wood-AL_Fixed_Double:wxh 20x05
  Tag: 108120
  Window Type Name: Window_Wood-AL_Fixed_Double:wxh 20x05

Window 3:
  GlobalId: 3V_$GkZXXCGeUhpQedgVIL
  Name: bona_window_sill:bona_window_sill:108122
  Description: None
  ObjectType: bona_window_sill:bona_window_sill
  Tag: 108122
  Window Type Name: bona_window_sill:bona_window_sill

Window 4:
  GlobalId: 2WPXLi24X7tu_zb5nE3qKQ
  Name: Skjutfönsterdörr:36x22 Nordan:173614
  Description: None
  ObjectType: Skjutfönsterdörr:36x22 Nordan
  Tag: 173614
  Window Type Name: Skjutfönsterdörr:36x22 Norda

In [18]:
# Explore window geometry and placement
if windows:
    window = windows[0]
    print("Geometry and Placement Information:")
    print("=" * 50)
    print(f"ObjectPlacement: {window.ObjectPlacement}")
    print(f"Representation: {window.Representation}")
    
    if window.Representation:
        for rep in window.Representation.Representations:
            print(f"\nRepresentation Type: {rep.RepresentationType}")
            print(f"RepresentationIdentifier: {rep.RepresentationIdentifier}")
            print(f"Items: {rep.Items}")

Geometry and Placement Information:
ObjectPlacement: #17405=IfcLocalPlacement(#17399,#17404)
Representation: #1930=IfcProductDefinitionShape($,$,(#1929))

Representation Type: MappedRepresentation
RepresentationIdentifier: Body
Items: (#1928=IfcMappedItem(#1902,#1927),)


In [19]:
# Explore materials used in window components
print("Materials Used in Windows:")
print("=" * 80)

for i, window in enumerate(windows, 1):
    print(f"\nWindow {i}: {window.Name}")
    print("-" * 80)
    
    # Check for material associations
    materials_found = False
    
    if hasattr(window, 'HasAssociations'):
        for association in window.HasAssociations:
            if association.is_a('IfcRelAssociatesMaterial'):
                materials_found = True
                relating_material = association.RelatingMaterial
                
                # Single material
                if relating_material.is_a('IfcMaterial'):
                    print(f"  Material: {relating_material.Name}")
                
                # Material layer set
                elif relating_material.is_a('IfcMaterialLayerSetUsage') or relating_material.is_a('IfcMaterialLayerSet'):
                    layer_set = relating_material.ForLayerSet if relating_material.is_a('IfcMaterialLayerSetUsage') else relating_material
                    print(f"  Material Layer Set: {layer_set.LayerSetName if hasattr(layer_set, 'LayerSetName') else 'Unnamed'}")
                    for layer in layer_set.MaterialLayers:
                        print(f"    - Layer: {layer.Material.Name}, Thickness: {layer.LayerThickness}")
                
                # Material list
                elif relating_material.is_a('IfcMaterialList'):
                    print(f"  Material List:")
                    for material in relating_material.Materials:
                        print(f"    - {material.Name}")
                
                # Material constituent set (for IFC4)
                elif relating_material.is_a('IfcMaterialConstituentSet'):
                    print(f"  Material Constituent Set: {relating_material.Name if relating_material.Name else 'Unnamed'}")
                    if hasattr(relating_material, 'MaterialConstituents'):
                        for constituent in relating_material.MaterialConstituents:
                            print(f"    - Component: {constituent.Name}, Material: {constituent.Material.Name}")
    
    if not materials_found:
        print("  No material information found")
    
    # Also check for materials in the window type
    if hasattr(window, 'IsDefinedBy'):
        for definition in window.IsDefinedBy:
            if definition.is_a('IfcRelDefinesByType'):
                window_type = definition.RelatingType
                if hasattr(window_type, 'HasAssociations'):
                    for type_association in window_type.HasAssociations:
                        if type_association.is_a('IfcRelAssociatesMaterial'):
                            print(f"\n  Materials from Window Type:")
                            relating_material = type_association.RelatingMaterial
                            
                            if relating_material.is_a('IfcMaterial'):
                                print(f"    Material: {relating_material.Name}")
                            elif relating_material.is_a('IfcMaterialList'):
                                for material in relating_material.Materials:
                                    print(f"    - {material.Name}")

Materials Used in Windows:

Window 1: Window_Wood-AL_Openable_Double:BxH 20x14 utan bleck:107943
--------------------------------------------------------------------------------
  Material List:
    - Bona_SE_Glass_Clear
    - <Unnamed>
    - Bona_Wood_Paint_WindowsInnerSide_Matte_White_NCSS0502Y
    - Bona_SE_Metal_Glossy
    - Bona_SE_Insulation_EPS

  Materials from Window Type:
    - Bona_SE_Glass_Clear
    - <Unnamed>
    - Bona_Wood_Paint_WindowsInnerSide_Matte_White_NCSS0502Y
    - Bona_SE_Metal_Glossy
    - Bona_SE_Insulation_EPS

Window 2: Window_Wood-AL_Fixed_Double:wxh 20x05:108120
--------------------------------------------------------------------------------
  Material List:
    - <Unnamed>
    - Bona_SE_Glass_Clear
    - Bona_Wood_Paint_WindowsInnerSide_Matte_White_NCSS0502Y
    - Bona_SE_Metal_Glossy
    - Bona_SE_Insulation_EPS

  Materials from Window Type:
    - <Unnamed>
    - Bona_SE_Glass_Clear
    - Bona_Wood_Paint_WindowsInnerSide_Matte_White_NCSS0502Y
    - Bon

In [27]:
# Get opacity of windows and their parts
print("Window Opacity Information:")
print("=" * 80)

def check_item_for_styles(item, indent="  "):
    """Recursively check an item and its nested items for style information"""
    styles_found = []
    
    # Get all IfcStyledItem entities that reference this item
    try:
        styled_items = ifc_file.get_inverse(item)
        for inverse_elem in styled_items:
            if inverse_elem.is_a('IfcStyledItem') and inverse_elem.Item == item:
                # Found a styled item
                if hasattr(inverse_elem, 'Styles'):
                    for style_assignment in inverse_elem.Styles:
                        if style_assignment.is_a('IfcPresentationStyleAssignment'):
                            for style in style_assignment.Styles:
                                if style.is_a('IfcSurfaceStyle'):
                                    style_name = style.Name if style.Name else 'Unnamed'
                                    for surface_style in style.Styles:
                                        if surface_style.is_a('IfcSurfaceStyleRendering'):
                                            transparency = surface_style.Transparency if hasattr(surface_style, 'Transparency') and surface_style.Transparency is not None else 0.0
                                            opacity = 1.0 - transparency
                                            styles_found.append({
                                                'style_name': style_name,
                                                'transparency': transparency,
                                                'opacity': opacity,
                                                'item_type': item.is_a(),
                                                'source': 'geometry'
                                            })
    except:
        pass
    
    # Recursively check nested items
    if item.is_a('IfcMappedItem'):
        mapping_source = item.MappingSource
        if mapping_source and hasattr(mapping_source, 'MappedRepresentation'):
            mapped_rep = mapping_source.MappedRepresentation
            if hasattr(mapped_rep, 'Items'):
                for mapped_item in mapped_rep.Items:
                    styles_found.extend(check_item_for_styles(mapped_item, indent))
    
    # Check boolean operands
    if item.is_a('IfcBooleanResult'):
        if hasattr(item, 'FirstOperand'):
            styles_found.extend(check_item_for_styles(item.FirstOperand, indent))
        if hasattr(item, 'SecondOperand'):
            styles_found.extend(check_item_for_styles(item.SecondOperand, indent))
    
    return styles_found

def get_material_styles(material):
    """Get styles from material definition representations"""
    styles_found = []
    
    # Check if material has representation
    try:
        material_reps = ifc_file.get_inverse(material)
        for inverse_elem in material_reps:
            if inverse_elem.is_a('IfcMaterialDefinitionRepresentation') and inverse_elem.RepresentedMaterial == material:
                # Found material definition representation
                for rep in inverse_elem.Representations:
                    if rep.is_a('IfcStyledRepresentation'):
                        for item in rep.Items:
                            if item.is_a('IfcStyledItem'):
                                for style_assignment in item.Styles:
                                    if style_assignment.is_a('IfcPresentationStyleAssignment'):
                                        for style in style_assignment.Styles:
                                            if style.is_a('IfcSurfaceStyle'):
                                                style_name = style.Name if style.Name else 'Unnamed'
                                                for surface_style in style.Styles:
                                                    if surface_style.is_a('IfcSurfaceStyleRendering'):
                                                        transparency = surface_style.Transparency if hasattr(surface_style, 'Transparency') and surface_style.Transparency is not None else 0.0
                                                        opacity = 1.0 - transparency
                                                        styles_found.append({
                                                            'style_name': style_name,
                                                            'material_name': material.Name if material.Name else 'Unnamed',
                                                            'transparency': transparency,
                                                            'opacity': opacity,
                                                            'source': 'material'
                                                        })
    except:
        pass
    
    return styles_found

for i, window in enumerate(windows, 1):
    print(f"\nWindow {i}: {window.Name}")
    print("-" * 80)
    
    all_styles = []
    
    # Check window representation for styled items
    if window.Representation:
        for rep in window.Representation.Representations:
            for item in rep.Items:
                styles = check_item_for_styles(item)
                all_styles.extend(styles)
    
    # Check materials for style information
    if hasattr(window, 'HasAssociations'):
        for association in window.HasAssociations:
            if association.is_a('IfcRelAssociatesMaterial'):
                relating_material = association.RelatingMaterial
                
                # Check single material
                if relating_material.is_a('IfcMaterial'):
                    material_styles = get_material_styles(relating_material)
                    all_styles.extend(material_styles)
                
                # Check material list
                elif relating_material.is_a('IfcMaterialList'):
                    for material in relating_material.Materials:
                        material_styles = get_material_styles(material)
                        all_styles.extend(material_styles)
                
                # Check material layer set
                elif relating_material.is_a('IfcMaterialLayerSetUsage') or relating_material.is_a('IfcMaterialLayerSet'):
                    layer_set = relating_material.ForLayerSet if relating_material.is_a('IfcMaterialLayerSetUsage') else relating_material
                    for layer in layer_set.MaterialLayers:
                        material_styles = get_material_styles(layer.Material)
                        all_styles.extend(material_styles)
    
    if all_styles:
        # Group by style name and show unique styles
        seen_styles = {}
        for style_info in all_styles:
            if style_info['source'] == 'geometry':
                style_key = (style_info['style_name'], style_info['transparency'], 'geometry')
            else:
                style_key = (style_info['style_name'], style_info['transparency'], style_info['material_name'])
            
            if style_key not in seen_styles:
                seen_styles[style_key] = style_info
        
        for style_info in seen_styles.values():
            if style_info['source'] == 'geometry':
                print(f"  Style: {style_info['style_name']} (from geometry)")
                print(f"    Item Type: {style_info['item_type']}")
            else:
                print(f"  Style: {style_info['style_name']} (from material: {style_info['material_name']})")
            print(f"    Transparency: {style_info['transparency']}")
            print(f"    Opacity: {style_info['opacity']}")
    else:
        print("  No opacity/transparency information found")

Window Opacity Information:

Window 1: Window_Wood-AL_Openable_Double:BxH 20x14 utan bleck:107943
--------------------------------------------------------------------------------
  Style: Bona_SE_Glass_Clear (from geometry)
    Item Type: IfcFacetedBrep
    Transparency: 0.91
    Opacity: 0.08999999999999997
  Style: Bona_SE_Metal_Glossy (from geometry)
    Item Type: IfcFacetedBrep
    Transparency: 0.0
    Opacity: 1.0
  Style: Bona_Wood_Paint_WindowsInnerSide_Matte_White_NCSS0502Y (from geometry)
    Item Type: IfcFacetedBrep
    Transparency: 0.0
    Opacity: 1.0
  Style: Bona_SE_Insulation_EPS (from geometry)
    Item Type: IfcFacetedBrep
    Transparency: 0.0
    Opacity: 1.0
  Style: Bona_SE_Glass_Clear (from material: Bona_SE_Glass_Clear)
    Transparency: 0.91
    Opacity: 0.08999999999999997
  Style: Bona_Wood_Paint_WindowsInnerSide_Matte_White_NCSS0502Y (from material: Bona_Wood_Paint_WindowsInnerSide_Matte_White_NCSS0502Y)
    Transparency: 0.0
    Opacity: 1.0
  Style: Bon

In [29]:
walls_without_windows

[#17157=IfcWallStandardCase('0XVYVfoMX509vqLOItOYQv',#18,'Basic Wall:Wall 1:289234',$,'Basic Wall:Wall 1',#17145,#17149,'289234'),
 #17173=IfcWallStandardCase('0XVYVfoMX509vqLOItOYN_',#18,'Basic Wall:Wall 1:289429',$,'Basic Wall:Wall 1',#17161,#17165,'289429'),
 #17203=IfcWallStandardCase('0XVYVfoMX509vqLOItOYNs',#18,'Basic Wall:Wall 1:289437',$,'Basic Wall:Wall 1',#17191,#17195,'289437')]